In [ ]:
from sklearn.preprocessing import OneHotEncoder
# load data
# define each column transformation:

global training___data
global validation_data

def drop(column_name, data_set):
    training___data.drop(column_name, axis=1)
    validation_data.drop(column_name, axis=1)

def MSZoning(dataSet):
    #1hot
    #anything with C(all) gets 1s for each
    #delete C(all)

def transformations(dataSet):
    return {
        "MSSubClass": drop("MSSubClass"),
        "MSZoning": MSZoning
    }
 

## 2do

- auto-build a template:
  - should have the useful functions (drop, one_hot, etc)
  - generate a "transformations" function that returns a dict with a key for each column that calls drop